In [34]:
import numpy as np
import pandas as pd
import os
os.chdir('/Users/toshan/dev/pysal/pysal/contrib/spint')
from gravity import Gravity, Production, Attraction, Doubly, BaseGravity
import statsmodels.formula.api as smf
from statsmodels.api import families
import pysal.spreg.ml_lag as ml_lag
import pysal

os.chdir('/Users/toshan/dev/pysal/pysal/weights')
from spintW import ODW

In [1043]:
austria = pd.read_csv('http://dl.dropbox.com/u/8649795/AT_Austria.csv')
#austria = austria[austria['Origin'] != austria['Destination']]
f = austria['Data'].values
o = austria['Origin'].values
d = austria['Destination'].values
dij = austria['Dij'].values
o_vars = austria['Oi2007'].values
d_vars = austria['Dj2007'].values

In [1038]:
o = pysal.weights.lat2W(3,3)
d = pysal.weights.lat2W(3,3, rook=False)
ODw = ODW(o,d)

In [1044]:
y = f.reshape((-1,1))
x = np.hstack([o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])

In [1045]:
model = ml_lag.ML_Lag(y, x, ODw)

In [1046]:
model.betas

array([[ -5.45317503e+02],
       [  8.49628168e-02],
       [  8.41943053e-02],
       [ -3.23718991e+00],
       [  4.91637163e-01]])

In [1030]:
model.vm1.shape

(6, 6)

In [1085]:
def m_inverse_prod(w, data, scalar, post_multiply=False, inv_method="power_exp", threshold=0.0000000001, max_iterations=None):
    """ 

    Parameters
    ----------

    w               : Pysal W object
                      nxn Pysal spatial weights object 

    data            : Numpy array
                      nx1 vector of data

    scalar          : float
                      Scalar value (typically rho or lambda)

    post_multiply   : boolean
                      If True then post-multiplies the data vector by the
                      inverse of the spatial filter, if false then
                      pre-multiplies.
    
    threshold       : float
                      Test value to stop the iterations. Test is against
                      sqrt(increment' * increment), where increment is a
                      vector representing the contribution from each
                      iteration.

    max_iterations  : integer
                      Maximum number of iterations for the expansion.   

    Examples
    --------
    """
    matrix = la.inv(np.eye(625))
    try:
        for i in range(len(w)):
            matrix -=  (scalar[i] * w[i].full()[0])
    except:
        for i in range(len(w)):
            matrix -= (scalar[i] * w[i])
    if post_multiply:
        inv_prod = spdot(data.T, np.asarray(matrix))
    else:
        inv_prod = spdot(np.asarray(matrix), data)
    return inv_prod




import numpy as np
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse.linalg import splu as SuperLU
import pysal as ps
from pysal.spreg.utils import RegressionPropsY, RegressionPropsVM, inverse_prod, spdot
import pysal.spreg.diagnostics as DIAG
import pysal.spreg.user_output as USER
import pysal.spreg.summary_output as SUMMARY
from pysal.spreg.w_utils import symmetrize
try:
    from scipy.optimize import minimize_scalar
    from scipy.optimize import minimize
    minimize_scalar_available = True
except ImportError:
    minimize_scalar_available = False

__all__ = ["ML_Lag"]


class BaseML_Lag(RegressionPropsY, RegressionPropsVM):

    """
    ML estimation of the spatial lag model (note no consistency
    checks, diagnostics or constants added); Anselin (1988) [Anselin1988]_
    Parameters
    ----------
    y            : array
                   nx1 array for dependent variable
    x            : array
                   Two dimensional array with n rows and one column for each
                   independent (exogenous) variable, excluding the constant
    w            : pysal W object
                   Spatial weights object
    method       : string
                   if 'full', brute force calculation (full matrix expressions)
                   if 'ord', Ord eigenvalue method
                   if 'LU', LU sparse matrix decomposition
    epsilon      : float
                   tolerance criterion in mimimize_scalar function and inverse_product
    Attributes
    ----------
    betas        : array
                   (k+1)x1 array of estimated coefficients (rho first)
    rho          : float
                   estimate of spatial autoregressive coefficient
    u            : array
                   nx1 array of residuals
    predy        : array
                   nx1 array of predicted y values
    n            : integer
                   Number of observations
    k            : integer
                   Number of variables for which coefficients are estimated
                   (including the constant, excluding the rho)
    y            : array
                   nx1 array for dependent variable
    x            : array
                   Two dimensional array with n rows and one column for each
                   independent (exogenous) variable, including the constant
    method       : string
                   log Jacobian method
                   if 'full': brute force (full matrix computations)
                   if 'ord' : Ord eigenvalue method
    epsilon      : float
                   tolerance criterion used in minimize_scalar function and inverse_product
    mean_y       : float
                   Mean of dependent variable
    std_y        : float
                   Standard deviation of dependent variable
    vm           : array
                   Variance covariance matrix (k+1 x k+1)
    vm1          : array
                   Variance covariance matrix (k+2 x k+2) includes sigma2
    sig2         : float
                   Sigma squared used in computations
    logll        : float
                   maximized log-likelihood (including constant terms)
    predy_e      : array
                   predicted values from reduced form
    e_pred       : array
                   prediction errors using reduced form predicted values
    Examples
    --------
    >>> import numpy as np
    >>> import pysal as ps
    >>> db =  ps.open(ps.examples.get_path("baltim.dbf"),'r')
    >>> ds_name = "baltim.dbf"
    >>> y_name = "PRICE"
    >>> y = np.array(db.by_col(y_name)).T
    >>> y.shape = (len(y),1)
    >>> x_names = ["NROOM","NBATH","PATIO","FIREPL","AC","GAR","AGE","LOTSZ","SQFT"]
    >>> x = np.array([db.by_col(var) for var in x_names]).T
    >>> x = np.hstack((np.ones((len(y),1)),x))
    >>> ww = ps.open(ps.examples.get_path("baltim_q.gal"))
    >>> w = ww.read()
    >>> ww.close()
    >>> w.transform = 'r'
    >>> w_name = "baltim_q.gal"
    >>> mllag = BaseML_Lag(y,x,w,method='ord') #doctest: +SKIP
    >>> "{0:.6f}".format(mllag.rho) #doctest: +SKIP
    '0.425885'
    >>> np.around(mllag.betas, decimals=4) #doctest: +SKIP
    array([[ 4.3675],
           [ 0.7502],
           [ 5.6116],
           [ 7.0497],
           [ 7.7246],
           [ 6.1231],
           [ 4.6375],
           [-0.1107],
           [ 0.0679],
           [ 0.0794],
           [ 0.4259]])
    >>> "{0:.6f}".format(mllag.mean_y) #doctest: +SKIP
    '44.307180'
    >>> "{0:.6f}".format(mllag.std_y) #doctest: +SKIP
    '23.606077'
    >>> np.around(np.diag(mllag.vm1), decimals=4) #doctest: +SKIP
    array([  23.8716,    1.1222,    3.0593,    7.3416,    5.6695,    5.4698,
              2.8684,    0.0026,    0.0002,    0.0266,    0.0032,  220.1292])
    >>> np.around(np.diag(mllag.vm), decimals=4) #doctest: +SKIP
    array([ 23.8716,   1.1222,   3.0593,   7.3416,   5.6695,   5.4698,
             2.8684,   0.0026,   0.0002,   0.0266,   0.0032])
    >>> "{0:.6f}".format(mllag.sig2) #doctest: +SKIP
    '151.458698'
    >>> "{0:.6f}".format(mllag.logll) #doctest: +SKIP
    '-832.937174'
    >>> mllag = BaseML_Lag(y,x,w) #doctest: +SKIP
    >>> "{0:.6f}".format(mllag.rho) #doctest: +SKIP
    '0.425885'
    >>> np.around(mllag.betas, decimals=4) #doctest: +SKIP
    array([[ 4.3675],
           [ 0.7502],
           [ 5.6116],
           [ 7.0497],
           [ 7.7246],
           [ 6.1231],
           [ 4.6375],
           [-0.1107],
           [ 0.0679],
           [ 0.0794],
           [ 0.4259]])
    >>> "{0:.6f}".format(mllag.mean_y) #doctest: +SKIP
    '44.307180'
    >>> "{0:.6f}".format(mllag.std_y) #doctest: +SKIP
    '23.606077'
    >>> np.around(np.diag(mllag.vm1), decimals=4) #doctest: +SKIP
    array([  23.8716,    1.1222,    3.0593,    7.3416,    5.6695,    5.4698,
              2.8684,    0.0026,    0.0002,    0.0266,    0.0032,  220.1292])
    >>> np.around(np.diag(mllag.vm), decimals=4) #doctest: +SKIP
    array([ 23.8716,   1.1222,   3.0593,   7.3416,   5.6695,   5.4698,
             2.8684,   0.0026,   0.0002,   0.0266,   0.0032])
    >>> "{0:.6f}".format(mllag.sig2) #doctest: +SKIP
    '151.458698'
    >>> "{0:.6f}".format(mllag.logll) #doctest: +SKIP
    '-832.937174'
    """

    def __init__(self, y, x, wo, wd, ww, method='full', epsilon=0.0000001):
        # set up main regression variables and spatial filters
        self.y = y
        self.x = x
        self.n, self.k = self.x.shape
        self.method = method
        self.epsilon = epsilon
        #W = w.full()[0]
        #Wsp = w.sparse
        oylag = ps.lag_spatial(wo, y)
        dylag = ps.lag_spatial(wd, y)
        wylag = ps.lag_spatial(ww, y)
        # b0, b1, e0 and e1
        xtx = spdot(self.x.T, self.x)
        xtxi = la.inv(xtx)
        xty = spdot(self.x.T, self.y)
        xtylo = spdot(self.x.T, oylag)
        xtyld = spdot(self.x.T, dylag)
        xtylw = spdot(self.x.T, wylag)
        b0 = np.dot(xtxi, xty)
        b1 = np.dot(xtxi, xtylo)
        b2 = np.dot(xtxi, xtyld)
        b3 = np.dot(xtxi, xtylw)
        e0 = self.y - spdot(x, b0)
        e1 = oylag - spdot(x, b1)
        e2 = dylag - spdot(x, b2)
        e3 = wylag - spdot(x, b3)
        methodML = method.upper()
        # call minimizer using concentrated log-likelihood to get rho
        if methodML in ['FULL', 'LU', 'ORD']:
            if methodML == 'FULL':
                Wo = wo.full()[0]     # moved here
                Wd = wd.full()[0]
                Ww = ww.full()[0]
                x0 = np.array([0.0, 0.0, 0.0])
                bounds = [(-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0)]
                res = minimize(lag_c_loglik, x0 , bounds=bounds,
                                      args=(
                                          self.n, e0, e1, e2, e3, Wo, Wd, Ww),
                                      tol=epsilon)
            elif methodML == 'LU':
                I = sp.identity(w.n)
                Wsp = w.sparse  # moved here
                res = minimize_scalar(lag_c_loglik_sp, 0.0, bounds=(-1.0,1.0),
                                      args=(self.n, e0, e1, I, Wsp),
                                      method='bounded', tol=epsilon)
            elif methodML == 'ORD':
                # check on symmetry structure
                if w.asymmetry(intrinsic=False) == []:
                    ww = symmetrize(w)
                    WW = ww.todense()
                    evals = la.eigvalsh(WW)
                else:
                    W = w.full()[0]     # moved here
                    evals = la.eigvals(W)
                res = minimize_scalar(lag_c_loglik_ord, 0.0, bounds=(-1.0, 1.0),
                                      args=(
                                          self.n, e0, e1, evals), method='bounded',
                                      tol=epsilon)
        else:
            # program will crash, need to catch
            print("{0} is an unsupported method".format(methodML))
            self = None
            return
        
        print res.x
        self.o_rho, self.d_rho, self.w_rho = res.x

        # compute full log-likelihood, including constants
        ln2pi = np.log(2.0 * np.pi)
        llik = -res.fun - self.n / 2.0 * ln2pi - self.n / 2.0
        self.logll = llik[0][0]

        # b, residuals and predicted values

        b = b0 - self.o_rho * b1 - self.d_rho * b2 - self.w_rho *b3
        self.betas = np.vstack((b, self.o_rho, self.d_rho, self.w_rho))   # rho added as last coefficient
        self.u = e0 - self.o_rho * e1 - self.o_rho * e2 - self.w_rho * e3
        self.predy = self.y - self.u
        xb = spdot(x, b)

        self.predy_e = m_inverse_prod(
            [wo.sparse, wd.sparse, ww.sparse], xb, [self.o_rho, self.d_rho, self.w_rho], inv_method="power_exp", threshold=epsilon)
        self.e_pred = self.y - self.predy_e

        # residual variance
        self.sig2 = self.sig2n  # no allowance for division by n-k
        print self.betas
        # information matrix
        a = -self.rho * W 
        np.fill_diagonal(a, 1.0)
        ai = la.inv(a)
        wai = np.dot(W, ai)
        tr1 = np.trace(wai)

        wai2 = np.dot(wai, wai)
        tr2 = np.trace(wai2)

        waiTwai = np.dot(wai.T, wai)
        tr3 = np.trace(waiTwai)

        wpredy = ps.lag_spatial(w, self.predy_e)
        wpyTwpy = np.dot(wpredy.T, wpredy)
        xTwpy = spdot(x.T, wpredy)
        
   

        # order of variables is beta, rho, sigma2

        v1 = np.vstack(
            (xtx / self.sig2, xTwpy.T / self.sig2, np.zeros((1, self.k))))
        v2 = np.vstack(
            (xTwpy / self.sig2, tr2 + tr3 + wpyTwpy / self.sig2, tr1 / self.sig2))
        v3 = np.vstack(
            (np.zeros((self.k, 1)), tr1 / self.sig2, self.n / (2.0 * self.sig2 ** 2)))

        v = np.hstack((v1, v2, v3))

        self.vm1 = la.inv(v)  # vm1 includes variance for sigma2
        self.vm = self.vm1[:-1, :-1]  # vm is for coefficients only


def lag_c_loglik(rho, n, e0, e1, e2, e3, Wo, Wd, Ww):
    # concentrated log-lik for lag model, no constants, brute force
    o_rho, d_rho, w_rho = rho
    er = e0 - o_rho * e1 - d_rho * e2 - w_rho * e3
    sig2 = np.dot(er.T, er) / n
    nlsig2 = (n / 2.0) * np.log(sig2)
    a = -o_rho * Wo + -d_rho * Wd + -w_rho * Ww
    np.fill_diagonal(a, 1.0)
    jacob = np.log(np.linalg.det(a))
    # this is the negative of the concentrated log lik for minimization
    clik = nlsig2 - jacob
    print clik
    return clik

def lag_c_loglik_sp(rho, n, e0, e1, I, Wsp):
    # concentrated log-lik for lag model, sparse algebra
    if isinstance(rho, np.ndarray):
        if rho.shape == (1,1):
            rho = rho[0][0] #why does the interior value change?
    er = e0 - rho * e1
    sig2 = np.dot(er.T, er) / n
    nlsig2 = (n / 2.0) * np.log(sig2)
    a = I - rho * Wsp
    LU = SuperLU(a.tocsc())
    jacob = np.sum(np.log(np.abs(LU.U.diagonal())))
    clike = nlsig2 - jacob
    return clike

def lag_c_loglik_ord(rho, n, e0, e1, evals):
    # concentrated log-lik for lag model, no constants, Ord eigenvalue method
    er = e0 - rho * e1
    sig2 = np.dot(er.T, er) / n
    nlsig2 = (n / 2.0) * np.log(sig2)
    revals = rho * evals
    jacob = np.log(1 - revals).sum()
    if isinstance(jacob, complex):
        jacob = jacob.real
    # this is the negative of the concentrated log lik for minimization
    clik = nlsig2 - jacob
    return clik



In [1032]:
model = ml_lag.BaseML_Lag(y, x, ODw)

In [1033]:
model.betas

array([[ 0.07543438],
       [ 0.08182455],
       [-4.43595233],
       [ 0.4109249 ]])

In [1047]:
import pysal.weights.spatial_lag as slag

o = pysal.weights.lat2W(3,3)
d = pysal.weights.lat2W(3,3, rook=False)
ODw = ODW(o,d)

wo = np.kron(o.full()[0], np.identity(9))
wo = pysal.weights.full2W(wo)
wd = np.kron(np.identity(9), d.full()[0])
wd = pysal.weights.full2W(wd)
             

AttributeError: 'numpy.ndarray' object has no attribute 'full'

In [1001]:
model = BaseML_Lag(y, x, wo, wd, ODw)

[[ 626.10223615]]
[[ 626.10223656]]
[[ 626.10223673]]
[[ 626.1022361]]
[[ 656.14509261]]
[[ 656.14509409]]
[[ 656.14509164]]
[[ 656.14509276]]
[[ 632.03389108]]
[[ 632.03388478]]
[[ 632.03388643]]
[[ 632.03389198]]
[[ 620.40771761]]
[[ 620.4077176]]
[[ 620.40771765]]
[[ 620.40771758]]
[[ 620.36518243]]
[[ 620.3651824]]
[[ 620.36518241]]
[[ 620.3651824]]
[[ 620.33686205]]
[[ 620.33686203]]
[[ 620.33686202]]
[[ 620.33686203]]
[[ 620.19916295]]
[[ 620.19916295]]
[[ 620.19916288]]
[[ 620.19916294]]
[[ 620.08649179]]
[[ 620.08649177]]
[[ 620.08649173]]
[[ 620.08649178]]
[[ 620.1121476]]
[[ 620.11214768]]
[[ 620.11214762]]
[[ 620.1121476]]
[[ 620.0723115]]
[[ 620.07231152]]
[[ 620.07231147]]
[[ 620.0723115]]
[[ 620.05913915]]
[[ 620.05913916]]
[[ 620.05913914]]
[[ 620.05913916]]
[[ 620.05796288]]
[[ 620.05796288]]
[[ 620.05796288]]
[[ 620.05796288]]
[[ 620.0579486]]
[[ 620.0579486]]
[[ 620.0579486]]
[[ 620.0579486]]
[-0.08135701 -0.09878033  0.32672886]
[[ 0.13567448]
 [ 0.11500647]
 [-5.492

AttributeError: 'BaseML_Lag' object has no attribute 'rho'

In [783]:
Wo = wo.full()[0]
Wd = wd.full()[0]
Ww = ODw.full()[0]
w = np.vstack([Wo, Wd, Ww])
a = -.5 * Wo
b = -.5 * Wd
c = -.5 * Ww

np.fill_diagonal(a, 1.0)
np.fill_diagonal(b, 1.0)
np.fill_diagonal(c, 1.0)

ai = la.inv(a)
bi = la.inv(b)
ci = la.inv(c)

np.vstack()

wai = np.dot(w, ai)
tr1 = np.trace(wai)

TypeError: vstack() takes exactly 1 argument (0 given)

In [ ]:
import numpy as np
from scipy.optimize import approx_fprime


def lag_c_loglik(rho, n, e0, e1, e2, e3, Wo, Wd, Ww):
    # concentrated log-lik for lag model, no constants, brute force
    o_rho, d_rho, w_rho = rho
    er = e0 - o_rho * e1 - d_rho * e2 - w_rho * e3
    sig2 = np.dot(er.T, er) / n
    nlsig2 = (n / 2.0) * np.log(sig2)
    a = -o_rho * Wo + -d_rho * Wd + -w_rho * Ww
    np.fill_diagonal(a, 1.0)
    jacob = np.log(np.linalg.det(a))
    # this is the negative of the concentrated log lik for minimization
    clik = nlsig2 - jacob
    return clik

def hessian ( x0, epsilon=1.e-5, linear_approx=False, *args ):
    """
    A numerical approximation to the Hessian matrix of cost function at
    location x0 (hopefully, the minimum)
    """
    # ``calculate_cost_function`` is the cost function implementation
    # The next line calculates an approximation to the first
    # derivative
    f1 = approx_fprime( x0, lag_c_loglik, *args) 

    # This is a linear approximation. Obviously much more efficient
    # if cost function is linear
    if linear_approx:
        f1 = np.matrix(f1)
        return f1.transpose() * f1    
    # Allocate space for the hessian
    n = x0.shape[0]
    hessian = np.zeros ( ( n, n ) )
    # The next loop fill in the matrix
    xx = x0
    for j in xrange( n ):
        xx0 = xx[j] # Store old value
        xx[j] = xx0 + epsilon # Perturb with finite difference
        # Recalculate the partial derivatives for this new point
        f2 = approx_fprime( x0, lag_c_loglik, *args) 
        hessian[:, j] = (f2 - f1)/epsilon # scale...
        xx[j] = xx0 # Restore initial value of x0        
    return hessian

hessian(rho, *args=n, e0, e1, e2, e3, Wo, Wd, Ww)

In [784]:
a = zip([1,2], [3,4])

In [785]:
a

[(1, 3), (2, 4)]

In [786]:
zip(*a)

[(1, 2), (3, 4)]

In [1040]:
import pysal.weights.spatial_lag as slag

o = pysal.weights.lat2W(3,3)
d = pysal.weights.lat2W(3,3)

wo = np.kron(o.full()[0], np.identity(9))
wo = pysal.weights.full2W(wo)
wd = np.kron(np.identity(9), d.full()[0])
wd = pysal.weights.full2W(wd)

ODw = ODW(o,d)

o_rho = 0
d_rho = 0
w_rho = 0

ar = np.linalg.inv(np.identity(81) - o_rho*wo.full()[0] + d_rho*wd.full()[0] - w_rho*ODw.full()[0])

In [808]:
betas = np.vstack([5,2,3,-.5])
X = np.hstack([np.ones((81,1)), x])

XB = np.dot(X,betas)

In [809]:
y = np.dot(ar,XB) * (XB + np.random.normal(0,500, (81,1)))

In [810]:
y.shape

(81, 1)

In [811]:
y

array([[  6.37093741e+08],
       [  8.16525403e+09],
       [  8.97635629e+09],
       [  4.68214332e+08],
       [  1.25760644e+09],
       [  1.16570361e+09],
       [  5.29762196e+08],
       [  4.07096567e+08],
       [  2.14117475e+08],
       [  3.52280616e+09],
       [  1.54191879e+10],
       [  1.67152814e+10],
       [  3.15852297e+09],
       [  4.90947839e+09],
       [  4.68781830e+09],
       [  3.35252418e+09],
       [  2.95551179e+09],
       [  2.39754416e+09],
       [  5.97181522e+09],
       [  2.03924950e+10],
       [  2.15978863e+10],
       [  5.47061949e+09],
       [  7.77132692e+09],
       [  7.50099143e+09],
       [  5.75724304e+09],
       [  5.21073016e+09],
       [  4.46833206e+09],
       [  6.81231632e+08],
       [  8.40364515e+09],
       [  9.21870664e+09],
       [  5.26497967e+08],
       [  1.39392007e+09],
       [  1.25879357e+09],
       [  6.11584616e+08],
       [  4.70339906e+08],
       [  2.66015239e+08],
       [  1.14166987e+09],
 

In [812]:
model = BaseML_Lag(y, X, wo, wd, ODw)

[[ 1686.39964026]]
[[ 1686.39964004]]
[[ 1686.39964063]]
[[ 1686.39964041]]
[[ 1853.56189284]]
[[ 1846.17328289]]
[[ 1846.17448887]]
[[ 1834.6697142]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[ 0.  0.  0.]
[[ -2.69186729e+09]
 [  2.52062693e+05]
 [  4.13341347e+05]
 [ -2.02588808e+06]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]]


AttributeError: 'BaseML_Lag' object has no attribute 'rho'

In [813]:
slag1 = slag()

TypeError: 'module' object is not callable

In [700]:
y = o_rho*wo.full()[0] + d_rho*wd.full()[0] + w_rho*ODw.full()[0] + 75 + (x*betas.T) + np.random.normal(0,50, (81,1))

ValueError: operands could not be broadcast together with shapes (81,81) (81,3) 

In [695]:
x*betas.T

array([[  8.64000000e+003,   1.63560000e+004,  -5.00000000e-301],
       [  8.64000000e+003,   8.15070000e+004,  -5.15009225e+001],
       [  8.64000000e+003,   8.61300000e+004,  -4.21023330e+001],
       [  8.64000000e+003,   1.30620000e+004,  -1.10405967e+002],
       [  8.64000000e+003,   2.72070000e+004,  -6.60037400e+001],
       [  8.64000000e+003,   2.57310000e+004,  -1.07255907e+002],
       [  8.64000000e+003,   1.48890000e+004,  -1.23466652e+002],
       [  8.64000000e+003,   1.17690000e+004,  -1.95428055e+002],
       [  8.64000000e+003,   6.07800000e+003,  -2.52544769e+002],
       [  4.29560000e+004,   1.63560000e+004,  -5.15009225e+001],
       [  4.29560000e+004,   8.15070000e+004,  -5.00000000e-301],
       [  4.29560000e+004,   8.61300000e+004,  -2.28981360e+001],
       [  4.29560000e+004,   1.30620000e+004,  -1.08497370e+002],
       [  4.29560000e+004,   2.72070000e+004,  -6.49390860e+001],
       [  4.29560000e+004,   2.57310000e+004,  -7.03533355e+001],
       [  

In [913]:
o_vars = np.random.randint(100,1000, (625,1))
d_vars = np.random.randint(100,1000, (625,1))
dij = np.random.randint(10,100, (625,1))
X = np.hstack([o_vars, d_vars, dij])
betas = np.vstack([2,3,5])

In [1020]:
betas = np.vstack([.1,.1,-5])
XB = np.dot(x,betas)

In [1009]:
import pysal.weights.spatial_lag as slag

o = pysal.weights.lat2W(3,3)
d = pysal.weights.lat2W(3,3)

wo = np.kron(o.full()[0], np.identity(9))
wo = pysal.weights.full2W(wo)
wd = np.kron(np.identity(9), d.full()[0])
wd = pysal.weights.full2W(wd)

ODw = ODW(o,d)

o_rho = -.1
d_rho = -.1
w_rho = .5

ar = np.linalg.inv(np.identity(81) - o_rho*wo.full()[0] - d_rho*wd.full()[0] - w_rho*ODw.full()[0])

betas = np.vstack([.1,.1,-5])
XB = np.dot(x,betas)

y = np.dot(ar ,(XB.reshape((-1,1)) + np.random.normal(0,50, (81,1))))



model = BaseML_Lag(y, x, wo, wd, ODw)

[[ 551.28530422]]
[[ 551.28530628]]
[[ 551.28530629]]
[[ 551.28530343]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[ 0.  0.  0.]
[[ 0.08656192]
 [ 0.08730784]
 [-3.80003764]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


AttributeError: 'BaseML_Lag' object has no attribute 'rho'

In [990]:
y.shape

(81, 1)

In [985]:
y

array([[ 1176.77835764],
       [ 2327.28616379],
       [ 2773.79835613],
       [ -188.07774066],
       [  782.63712525],
       [  230.92912647],
       [ -143.52604269],
       [ -716.0031344 ],
       [-1516.23393291],
       [ 1940.80283463],
       [ 3805.32781305],
       [ 3907.46513873],
       [ 1121.57415529],
       [ 1144.77202716],
       [ 1739.01574574],
       [ 1471.8807629 ],
       [  527.9996231 ],
       [  321.30460011],
       [ 2938.1903554 ],
       [ 4401.81465504],
       [ 5321.37766243],
       [ 1485.6195754 ],
       [ 1715.00253399],
       [ 2082.30785041],
       [ 1988.47088777],
       [ 1092.59112419],
       [  822.93037061],
       [   11.55695528],
       [ 1594.02456514],
       [ 1694.41684272],
       [ 1058.41565794],
       [  506.3272004 ],
       [  392.72624321],
       [  402.58359362],
       [  -61.93500499],
       [ -392.95690462],
       [  730.52433536],
       [ 2010.63477333],
       [ 2342.07291894],
       [  568.54570922],


In [1098]:
austria = pd.read_csv('http://dl.dropbox.com/u/8649795/AT_Austria.csv')
#austria = austria[austria['Origin'] != austria['Destination']]
f = austria['Data'].values
o = austria['Origin'].values
d = austria['Destination'].values
#dij = austria['Dij'].values
#o_vars = austria['Oi2007'].values
#d_vars = austria['Dj2007'].values
dij = np.random.randint(100,1000, (625,1))
o_vars = np.random.randint(100,10000, (625,1))
d_vars = np.random.randint(100,10000, (625,1))
x = np.hstack([o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])

import pysal.weights.spatial_lag as slag

o = pysal.weights.lat2W(5,5)
d = pysal.weights.lat2W(5,5, rook=False)
ODw = ODW(o,d)

wo = np.kron(o.full()[0], np.identity(25))
wo = pysal.weights.full2W(wo)
wd = np.kron(np.identity(25), d.full()[0])
wd = pysal.weights.full2W(wd)

betas = np.vstack([.1,.1,-5])
XB = np.dot(x,betas)


o_rho = .4
d_rho = .4
w_rho = -.16

ar = np.linalg.inv(np.identity(625) - o_rho*wo.full()[0] - d_rho*wd.full()[0] - w_rho*ODw.full()[0])


y = np.dot(ar ,(XB.reshape((-1,1)) + np.random.normal(0,50, (625,1))))
model = BaseML_Lag(y, x, wo, wd, ODw)

[[ 5905.20966842]]
[[ 5905.20964995]]
[[ 5905.20967162]]
[[ 5905.20966931]]
[[ inf]]
[[ inf]]
[[ 7211.15225588]]
[[ inf]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[[ nan]]
[ 0.  0.  0.]
[[-0.09958943]
 [ 0.07076203]
 [ 1.63491934]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


AttributeError: 'BaseML_Lag' object has no attribute 'rho'

In [1096]:
model.betas

array([[ 0.15149156],
       [ 0.04000845],
       [ 0.32624137],
       [-0.42212776]])

In [1080]:
y

array([[  -14.95301022],
       [-1939.78522006],
       [-1563.98711212],
       [ -945.73877374],
       [-2339.93584798],
       [-2686.70960741],
       [  -65.4869049 ],
       [-2040.66664833],
       [ 1227.79946869],
       [-1432.63408639],
       [-1566.21597921],
       [  371.49978233],
       [-1983.78124497],
       [ -166.45900241],
       [  -87.69080408],
       [  225.94106622],
       [   54.63799603],
       [-1459.77912914],
       [  739.05634455],
       [-1532.90956039],
       [-2136.89751846],
       [  435.73259463],
       [ 1594.6454202 ],
       [ -404.50344936],
       [ -165.76271441],
       [-3180.70921134],
       [-1560.25609184],
       [-3561.84013958],
       [  869.70687937],
       [-3574.80487801],
       [-1399.04064192],
       [ 2697.28958954],
       [-3335.90075714],
       [  -55.25562241],
       [  220.71085666],
       [-3053.41185501],
       [ -908.46007919],
       [-3749.37282053],
       [ -615.09114671],
       [ 2294.77884602],
